In [1]:
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm


%load_ext autoreload
%autoreload 2

In [2]:
num_gt_matrix = np.load("num_gt_matrix.npy")
with open("num_gt_matrix_index.pl", 'rb') as f:
    num_gt_matrix_index = pickle.load(f)

In [38]:
models_paths = list(Path(".").glob("model_type=RF*"))
models_paths

[PosixPath('model_type=RF__env_window_size=10s__time_strategy=present__env_feats=ground-truth-v3__stride_strategy=window-size__other=condensed-k_10__te=0.47.pl'),
 PosixPath('model_type=RF__env_window_size=10s__time_strategy=present__env_feats=pipeline-output-v1__stride_strategy=window-size__other=condensed-k_10__te=-10290005.14.pl'),
 PosixPath('model_type=RF__env_window_size=4s__time_strategy=past-1-step__env_feats=pipeline-output-v1__stride_strategy=window-size__other=condensed-k_10__te=-12025128.71.pl'),
 PosixPath('model_type=RF__env_window_size=10s__time_strategy=past-1-step__env_feats=ground-truth-v3__stride_strategy=window-size__other=condensed-k_10__te=-40292943.22.pl'),
 PosixPath('model_type=RF__env_window_size=4__time_strategy=past-1-step__env_feats=ground-truth-v3__stride_strategy=uniform_window_skip__other=k_10-condensed-space__te=-636474595.77.pl')]

In [39]:
model_dicts = []
for p in models_paths:
    with open(p, 'rb') as f:
        model_dicts.append(pickle.load(f))

In [40]:
for m in model_dicts:
    print(m["metadata"])

{'model_type': 'RF', 'env_window_size': '10s', 'time_strategy': 'present', 'env_feats': 'ground-truth-v3', 'stride_strategy': 'window-size', 'other': 'condensed-k_10'}
{'model_type': 'RF', 'env_window_size': '10s', 'time_strategy': 'present', 'env_feats': 'pipeline-output-v1', 'stride_strategy': 'window-size', 'other': 'condensed-k_10'}
{'model_type': 'RF', 'env_window_size': '4s', 'time_strategy': 'past-1-step', 'env_feats': 'pipeline-output-v1', 'stride_strategy': 'window-size', 'other': 'condensed-k_10'}
{'model_type': 'RF', 'env_window_size': '10s', 'time_strategy': 'past-1-step', 'env_feats': 'ground-truth-v3', 'stride_strategy': 'window-size', 'other': 'condensed-k_10'}
{'model_type': 'RF', 'env_window_size': 4, 'time_strategy': 'past-1-step', 'env_feats': 'ground-truth-v3', 'stride_strategy': 'uniform_window_skip', 'other': 'k_10-condensed-space'}


In [6]:
from ad_config_search.utils import create_sequential_policy, contract_one_hot,\
        get_rows, evaluate_configuration_sequence, evaluate_fine_grained_policy

In [7]:
def highest_avg_config(exp_dict):
    """
    Computes only over training dataset as it should be.
    Only focuses on the configuration knobs ['D-model', 'D-conf',
    'D-seq-pol', 'T-min-iou', 'T-max-age', 'T-every-nth-det'] and the
    scenario columns for the sake of computing the average score. All
    other columns are ignored.
    """
    config_knobs = ['D-model', 'D-conf', 'D-seq-pol', 'T-min-iou', 'T-max-age', 'T-every-nth-det']
    train_copy = exp_dict["train"].copy()
    train_copy["score"] = exp_dict["train_labels"]
    train_copy["scenario_name"] = train_copy.index
    train_copy = contract_one_hot(train_copy)
    train_copy = train_copy.drop_duplicates(subset=config_knobs+["scenario_name"])  # essentially ignores other columns
    highest_tup = None
    config_score_pairs = \
        [(config_tup, rows["score"].mean()) for config_tup, rows in train_copy.groupby(config_knobs)]
    highest_tup, _ = sorted(config_score_pairs, key=lambda x: x[1])[-1]
    best_average_config = dict(zip(config_knobs, highest_tup))
    return best_average_config

In [8]:
def po_past_scores(exp_dict, num_gt_matrix, num_gt_matrix_index):
    best_average_config = highest_avg_config(exp_dict)
    
    test_copy = exp_dict["test"].copy()
    test_copy["scenario_name"] = test_copy.index
    test_copy["prediction"] = exp_dict["model"].predict(exp_dict["test"])
    test_copy = contract_one_hot(test_copy)
        
    final_policy = {} 
    # iterate over full scenarios
    for scenario in tqdm(test_copy["scenario_name"].map(lambda x: x.split("-P")[0]).unique()):
        scenario_policy = create_sequential_policy(scenario, best_average_config, test_copy)
        final_policy.update(scenario_policy)
        if scenario == "training_0002-S4":
            print(scenario_policy)

    return evaluate_fine_grained_policy(final_policy, num_gt_matrix, num_gt_matrix_index)

In [23]:
learned_po_past_4s_scores = po_past_scores(model_dicts[2], num_gt_matrix, num_gt_matrix_index)

[Parallel(n_jobs=144)]: Using backend ThreadingBackend with 144 concurrent workers.
[Parallel(n_jobs=144)]: Done 162 tasks      | elapsed:    0.3s
[Parallel(n_jobs=144)]: Done 512 tasks      | elapsed:    0.8s
[Parallel(n_jobs=144)]: Done 962 tasks      | elapsed:    1.5s
[Parallel(n_jobs=144)]: Done 1512 tasks      | elapsed:    2.6s
[Parallel(n_jobs=144)]: Done 2400 out of 2400 | elapsed:    3.8s finished
 18%|█▊        | 9/49 [00:06<00:29,  1.36it/s]

training_0000-S16-P1_5 doesn't exist in the data fed!
training_0000-S16-P2_5 doesn't exist in the data fed!
training_0000-S16-P3_5 doesn't exist in the data fed!
training_0000-S16-P4_5 doesn't exist in the data fed!


 51%|█████     | 25/49 [00:18<00:17,  1.37it/s]

training_0002-S9-P1_5 doesn't exist in the data fed!
training_0002-S9-P2_5 doesn't exist in the data fed!
training_0002-S9-P3_5 doesn't exist in the data fed!
training_0002-S9-P4_5 doesn't exist in the data fed!


 73%|███████▎  | 36/49 [00:26<00:09,  1.37it/s]

...

 86%|████████▌ | 42/49 [00:30<00:05,  1.35it/s]

training_0004-S20-P1_5 doesn't exist in the data fed!
training_0004-S20-P2_5 doesn't exist in the data fed!
training_0004-S20-P3_5 doesn't exist in the data fed!
training_0004-S20-P4_5 doesn't exist in the data fed!


 90%|████████▉ | 44/49 [00:32<00:03,  1.36it/s]

training_0004-S5-P1_5 doesn't exist in the data fed!
training_0004-S5-P2_5 doesn't exist in the data fed!
training_0004-S5-P3_5 doesn't exist in the data fed!
training_0004-S5-P4_5 doesn't exist in the data fed!


100%|██████████| 49/49 [00:07<00:00,  6.64it/s]


In [11]:
with open("learned_po_past_4s_scores.pl", 'wb') as f:
    pickle.dump(learned_po_past_4s_scores, f)

In [15]:
def gt_scores(exp_dict, num_gt_matrix, num_gt_matrix_index):
    config_knobs = ['D-model', 'D-conf', 'D-seq-pol', 'T-min-iou', 'T-max-age', 'T-every-nth-det']
    test_copy = exp_dict["test"].copy()
    test_copy["scenario_name"] = test_copy.index
    test_copy["prediction"] = exp_dict["model"].predict(exp_dict["test"])
    test_copy = contract_one_hot(test_copy)
    
    final_policy = {}
    for scenario, rows in test_copy.groupby(["scenario_name"]):
        best_row = rows.iloc[rows["prediction"].argmax()]
        best_config = {k: best_row[k] for k in config_knobs}
        final_policy[scenario] = best_config
    
    print(list(final_policy.keys())[0])
    return evaluate_fine_grained_policy(final_policy, num_gt_matrix, num_gt_matrix_index, default_config=highest_avg_config(exp_dict))

In [24]:
learned_gt_present_4s_scores = gt_scores(model_dicts[0], num_gt_matrix, num_gt_matrix_index)

[Parallel(n_jobs=144)]: Using backend ThreadingBackend with 144 concurrent workers.
[Parallel(n_jobs=144)]: Done 174 tasks      | elapsed:    0.2s
[Parallel(n_jobs=144)]: Done 524 tasks      | elapsed:    0.4s
[Parallel(n_jobs=144)]: Done 974 tasks      | elapsed:    0.6s
[Parallel(n_jobs=144)]: Done 1600 out of 1600 | elapsed:    0.9s finished


training_0000-S0-P0_5


 98%|█████████▊| 48/49 [00:07<00:00,  6.75it/s]

only found [1, 2, 3, 4] of scenario training_0005-S23


100%|██████████| 49/49 [00:07<00:00,  6.73it/s]


In [26]:
with open("learned_gt_present_4s_scores.pl", 'wb') as f:
    pickle.dump(learned_gt_present_4s_scores, f)

In [25]:
learned_gt_past_4s_scores = gt_scores(model_dicts[3], num_gt_matrix, num_gt_matrix_index)

[Parallel(n_jobs=144)]: Using backend ThreadingBackend with 144 concurrent workers.
[Parallel(n_jobs=144)]: Done 400 out of 400 | elapsed:    0.2s finished
  0%|          | 0/49 [00:00<?, ?it/s]

training_0000-S0-P1_5
only found [1, 2, 3, 4] of scenario training_0000-S0


  4%|▍         | 2/49 [00:00<00:06,  6.78it/s]

only found [1, 2, 3, 4] of scenario training_0000-S1
only found [1, 2, 3, 4] of scenario training_0000-S10


  8%|▊         | 4/49 [00:00<00:06,  6.85it/s]

...

 94%|█████████▍| 46/49 [00:06<00:00,  6.95it/s]

only found [1, 2, 3, 4] of scenario training_0005-S12
only found [1, 2, 3, 4] of scenario training_0005-S19


 98%|█████████▊| 48/49 [00:06<00:00,  6.94it/s]

only found [2, 3, 4] of scenario training_0005-S23
only found [1, 2, 3, 4] of scenario training_0005-S6


100%|██████████| 49/49 [00:07<00:00,  6.93it/s]


In [42]:
learned_gt_past_4s_scores = gt_scores(model_dicts[4], num_gt_matrix, num_gt_matrix_index)

[Parallel(n_jobs=143)]: Using backend ThreadingBackend with 143 concurrent workers.
[Parallel(n_jobs=143)]: Done 164 tasks      | elapsed:    0.2s
[Parallel(n_jobs=143)]: Done 514 tasks      | elapsed:    0.4s
[Parallel(n_jobs=143)]: Done 800 out of 800 | elapsed:    0.5s finished
  0%|          | 0/49 [00:00<?, ?it/s]

training_0000-S0-P1_5
only found [1, 2, 3, 4] of scenario training_0000-S0


  4%|▍         | 2/49 [00:00<00:06,  7.14it/s]

only found [1, 2, 3, 4] of scenario training_0000-S1
only found [1, 2, 3, 4] of scenario training_0000-S10


  8%|▊         | 4/49 [00:00<00:06,  6.94it/s]

...

 94%|█████████▍| 46/49 [00:06<00:00,  7.10it/s]

only found [1, 2, 3, 4] of scenario training_0005-S12
only found [1, 2, 3, 4] of scenario training_0005-S19


 98%|█████████▊| 48/49 [00:06<00:00,  7.10it/s]

only found [2, 3, 4] of scenario training_0005-S23
only found [1, 2, 3, 4] of scenario training_0005-S6


100%|██████████| 49/49 [00:07<00:00,  6.96it/s]


In [43]:
with open("learned_gt_past_4s_scores.pl", 'wb') as f:
    pickle.dump(learned_gt_past_4s_scores, f)

In [29]:
def po_present_scores(exp_dict, num_gt_matrix, num_gt_matrix_index):
    config_knobs = ['D-model', 'D-conf', 'D-seq-pol', 'T-min-iou', 'T-max-age', 'T-every-nth-det']
    test_copy = exp_dict["test"].copy()
    test_copy["scenario_name"] = test_copy.index
    test_copy["prediction"] = exp_dict["model"].predict(exp_dict["test"])
    test_copy = contract_one_hot(test_copy)
    
    final_min_max_policy = {}
    final_max_max_policy = {}
    for scenario, scenario_rows in tqdm(test_copy.groupby(["scenario_name"])):
        best_config_and_score = []
        for config_tup, config_rows in scenario_rows.groupby([f'pastC-{k}' for k in config_knobs]):
            best_row = config_rows.iloc[config_rows["prediction"].argmax()]
            best_config = {k: best_row[k] for k in config_knobs}
            best_score = best_row["prediction"]
            best_config_and_score.append((best_config, best_score))
        sorted_config_scores = sorted(best_config_and_score, key=lambda x: x[1])
        final_min_max_policy[scenario] = sorted_config_scores[0][0]
        final_max_max_policy[scenario] = sorted_config_scores[-1][0]
    
    min_max_policy_scores = evaluate_fine_grained_policy(final_min_max_policy, num_gt_matrix, num_gt_matrix_index, default_config=highest_avg_config(exp_dict))
    max_max_policy_scores = evaluate_fine_grained_policy(final_max_max_policy, num_gt_matrix, num_gt_matrix_index, default_config=highest_avg_config(exp_dict))
    return min_max_policy_scores, max_max_policy_scores

In [30]:
learned_po_present_min_max_4s_scores, learned_po_present_max_max_4s_scores = po_present_scores(model_dicts[1], num_gt_matrix, num_gt_matrix_index)

[Parallel(n_jobs=144)]: Using backend ThreadingBackend with 144 concurrent workers.
[Parallel(n_jobs=144)]: Done 400 out of 400 | elapsed:    0.7s finished
100%|██████████| 49/49 [00:06<00:00,  7.17it/s]


In [31]:
with open("learned_po_present_min_max_4s_scores.pl", 'wb') as f:
    pickle.dump(learned_po_present_min_max_4s_scores, f)
with open("learned_po_present_max_max_4s_scores.pl", 'wb') as f:
    pickle.dump(learned_po_present_max_max_4s_scores, f)

In [37]:
for m in model_dicts:
    print(m["metadata"])
    print(m["model"])

{'model_type': 'RF', 'env_window_size': '10s', 'time_strategy': 'present', 'env_feats': 'ground-truth-v3', 'stride_strategy': 'window-size', 'other': 'condensed-k_10'}
RandomForestRegressor(max_depth=24, max_features=4,
                      min_impurity_decrease=0.00022898035917795723,
                      n_estimators=1600, n_jobs=-1, random_state=50, verbose=1)
{'model_type': 'RF', 'env_window_size': '10s', 'time_strategy': 'present', 'env_feats': 'pipeline-output-v1', 'stride_strategy': 'window-size', 'other': 'condensed-k_10'}
RandomForestRegressor(max_depth=1, max_features=8,
                      min_impurity_decrease=0.06486178665526617,
                      n_estimators=400, n_jobs=-1, random_state=50, verbose=1)
{'model_type': 'RF', 'env_window_size': '4s', 'time_strategy': 'past-1-step', 'env_feats': 'pipeline-output-v1', 'stride_strategy': 'window-size', 'other': 'condensed-k_10'}
RandomForestRegressor(max_depth=1, max_features=6,
                      min_impurity_decrea